# Data Cleaning

Use **Code** cells to write and run any code you need to answer the question and **Markdown** cells to write out answers in words. After you are finished with the assignment, remember to download it as an **HTML file** and submit it in **ELMS**.

In [ ]:
from requests import get

import numpy as np 
import pandas as pd 

## Missing Values 

Typically one of the first things you'll want to do when exploring a new data set is checking for missing  values. This might seem fairly straightforward, but it can actually get quite complicated, and the way that you deal with missing values will be highly dependent on the context. There also isn't always a right or wrong answer in how you should deal with missing values. Sometimes, values are missing because of a data collection issue. Other times, it is because the respondent did not want to respond. It is important to think through what each of these scenarios might mean for our analysis, and try to make sure that our approach is properly representing the scenario. 

For example, suppose a survey gave only two options for a "Gender" variable: "Male" or "Female". If we observe a missing value, this could represent someone who didn't fit into one of those two categories, or it could be someone who did but didn't want to answer. Or, it could just be that the person missed the question or accidentally skipped over it. However, if we simply remove all rows with a missing "Gender" variable, then we are removing all people who do not identify as either "Male" or "Female", which can have implications for our analysis.

### Steps for Dealing with Missing Data

1. **Identify where, how much, and what type of missing data you have.** Sometimes, missing values will be easy to find because they will be coded as `None` or `np.NaN` or similar types of missing values. Other times, they will be harder to identify, because they might be coded as large negative numbers (even though the values are counts or percentages) or as empty strings (which might not show up strictly as missing, but just as a string with no characters). 
2. **Determine how you want to deal with missing values.** This can be on a case by case basis. For example, if you are making a graph with two variables, you might want to subset to those variables first, because missingness on other variables won't affect your graph. If you want to find summary statistics of numeric variables such as counts, you'll first need to make sure you remove any values that were coded as large negative values, for example.
3. **Use data cleaning steps to deal with missing values.** You might need to change certain values to `np.NaN` values or perhaps remove certain rows that aren't useful. You also might need to make decisions on how you should fix certain values.

## Starting out: Identifying Missing Values

Let's take a look at the Pulse of the Nation dataset. 

In [ ]:
data_file = '201807-CAH_PulseOfTheNation_Raw.csv'
df = pd.read_csv(data_file)
df.head()

The `.info` method can help identify the number of non-null values, and thus help identify which variables might have missing values.

In [ ]:
df.info()

In [ ]:
df.shape

Using `isnull()` gives us a data frame of the same size with `True` and `False` values depending on whether it was a missing value or not. Then, `sum()` sums each column. Since Python treats `True` as `1` and `False` as `0`, the sum of each column gives us the total number of missing values for each variable.

In [ ]:
df.isnull().sum()

<font color ='red'>**Question 1: What are the variables with missing values? Why might these be missing?**</font>

Answer:

## Data Transformation to Deal with Missing Values

There's lots of ways to deal with missing values. The first step in determining how to deal with them is looking at your dataset to figure out why they are missing and how this might affect your analysis.

### Method 1: Drop the Columns

Many times, you won't need to work with that column. Instead, you'll just need to work with other variables that may not have missing values. Working with just the specific columns that you need might help solve the issue. We've already done this previously with this dataset. Since only a few of the variables had missing values, we started by analyzing the ones that didn't. If you want to make sure that any columns with missing values were dropped, you can use `df.dropna(axis = 1)`. The `dropna` method drops any row or column with missing values. The `axis = 1` argument specifies that it should be the columns (if we had done `axis = 0`, then it would drop any rows with missing values).

In [ ]:
df_drop_na_cols = df.dropna(axis = 1)
df_drop_na_cols.head()

In [ ]:
df_drop_na_cols.shape

### Method 2: Remove any rows with missing values

We could also use the `dropna` method with the argument `axis = 0` to drop any rows with missing values. However, you do need to be careful when doing this. You might dropping many more observations than you want, because a row can be mostly complete, but will be dropped if there is any missing value. With the Pulse of the Nation dataset, we get 0 rows after doing this because a few variables have lots of missing values.

In [ ]:
df.dropna(axis = 0)

<font color ='red'>**Question 2: Why are there no rows without missing values? What about the dataset might have indicated this might happen?**</font>

Answer:

### Method 3: Fix the missing values

Many times, the issue with missing values can be addressed in other ways. This is where the context of the data is very important. In this example, most of the missing values are present in variables that are actually part of another variable. For example, the `race_other` variable was only answered if the respondent said `Other` for their Race. This is a case where we might combine the `race` and `race_other` variables, or we might decide to do something else entirely with that column. In order to decide what to do, let's take a look at what is inside the `race_other` variable.

In [ ]:
df.race.value_counts()

In [ ]:
df.race_other.value_counts()

In [ ]:
df.race.value_counts()

It looks like there's a mix of Native American/American Indian responses, Mixed Race responses, one Pacific Islander, as well as some others that might be grouped in with existing categories. Even though Latino and Hispanic are not the exact same, we might create a Hispanic/Latino category and include the one "Hispanic" response with that. We might also put the "BRITISH - CAUCASIAN" and "Europian" responses within "White" (even though "Europian" might not exactly be "White"). So, we can go in and adjust some of these responses to end up with just one overall `race` variable. 

We can use the `map` method to do this. 

In [ ]:
race_map = {'Mixed':'Mixed Race',
           'Mixed race': 'Mixed Race',
           'mixed race': 'Mixed Race',
           'American Indian':'Native American',
           'American-Indian':'Native American',
            'Native American':'Native American',
           'Hispanic':'Latino',
           'BRITISH - CAUCASIAN': 'White',
           'Europian': 'White',
           'Pacific Islander':'Pacific Islander'}
fixed_other_race = df.race_other.map(race_map)

In [ ]:
fixed_other_race.value_counts()

In [ ]:
df_new = df
df_new.loc[df_new.race == 'Other: SPECIFY','race'] = fixed_other_race[df.race == 'Other: SPECIFY']

In [ ]:
df_new.race.value_counts()

In [ ]:
df_clean = df_new.drop('race_other', axis = 1)
df_clean.head()

We also could have done this by manually changing these one by one. This works ok for this example because there are so few. In fact, this might be easier to do and keep track of with so few values to change.

In [ ]:
# Adding Native American responses
df.loc[df.race_other == 'Native American','race'] = 'Native American'
df.loc[df.race_other == 'American Indian','race'] = 'Native American'
df.loc[df.race_other == 'American-Indian','race'] = 'Native American'

In [ ]:
# Adding Mixed Race
df.loc[df.race_other == 'Mixed','race'] = 'Mixed Race'
df.loc[df.race_other == 'Mixed race','race'] = 'Mixed Race'
df.loc[df.race_other == 'mixed race','race'] = 'Mixed Race'

<font color ='red'>**Question 3: The start of some of the updating is done above. Change the rest of the values in the same way we did in the mapping dictionary.**</font>

We can also drop any duplicated rows, if we had any.

In [ ]:
df_no_dups = df.drop_duplicates()
df_no_dups.shape # Check how many rows there are after dropping duplicates

The other set of missing values in this dataset is in the `rep_change` and `dem_change` variables. These questions were only asked if the respondent said they were either Republican or Democrat. For the `rep_change` variable, the respondent had to have responded that they were Republican, and were asked if they would change parties if scientific evidence proved that they other party's policies generally improved the nation. The same was the case for `dem_change` for Democrats.

In [ ]:
df_clean.rep_change.value_counts()

In [ ]:
df_clean.dem_change.value_counts()

<font color ='red'>**Question 4: What would be the best way to deal with the missing values in these two variables? Should they combined in any way? Why or why not?**</font>

Answer:

## Missing Values not coded as missing

Recall that with the data we pulled from the Census API, some values were coded as large negative numbers. These are typically missing because the number of people in that area is too low to be reported. With the Census, this is usually for privacy reasons, since they have the data but don't want to reveal too much about individuals. With other groups, it might be because of lack of data or uncertainty about the estimates (for 1-year ACS data, uncertainty also plays a role). Some information about what some of these values might mean are included here: https://www.census.gov/data/developers/data-sets/acs-1year/notes-on-acs-estimate-and-annotation-values.html

In any case, these are values that are missing, but aren't coded as `np.NaN` or something that would show up easily. Let's take a look at an ACS dataset to see what we mean.

In [ ]:
with open('census-key.txt', 'r') as f:
    census_key = f.readline()

census_base_url = 'https://api.census.gov/data/2022/acs/acs1/profile'

census_params = {'get':'NAME,DP03_0087E,DP05_0037PE,DP05_0038PE,DP05_0039PE,DP05_0044PE,DP05_0073PE',
                 'for':'county:*',
                 'key':census_key}

r = get(census_base_url, params = census_params)
r.status_code

In [ ]:
counties = r.json()
counties[:3]

In [ ]:
# Get the data into dictionary format
keys = ['county', 'mean_income', 'percent_white','percent_black','percent_indian','percent_asian','percent_hispanic']
county_dict = {key:[county[keys.index(key)] for county in counties[1:]] for key in keys}

In [ ]:
county_df = pd.DataFrame(county_dict)
county_df.head()

In [ ]:
# Change columns that are numeric to numeric
county_df[keys[1:]] = county_df[keys[1:]].apply(pd.to_numeric, axis = 1)

In [ ]:
county_df.head()

Let's take a look at the `info` for `county_df`. 

In [ ]:
county_df.info()

This makes it look like there are no missing values. However, if you were to use `describe` ...

In [ ]:
county_df.describe()

The minimum value for many of these variables is below 0. However, we know this can't be the case because these are percentages. Let's take a look at what those rows might look like.

In [ ]:
county_df[county_df.percent_white < 0]

<font color ='red'>**Question 5: How would you deal with these missing values? What would be the best way to code these?.**</font>

Once we decide on how we want to deal with these values, we can use the `replace` method to fill in the missing values. As an example, we replace all values of `-999999999.0` with `np.NaN`. This makes it so that those values are coded as missing.

In [ ]:
county_df_filled = county_df.replace(-999999999.0, np.nan)
county_df_filled[county_df_filled.percent_white.isnull()]

In [ ]:
county_df_filled.describe()

What if we wanted to put 0 instead? Remember, these are values that are missing because there were too few people. It might make sense to essentially round down to 0. We can use the `fillna` method to do this.

In [ ]:
county_zero_filled = county_df_filled.fillna(0)
county_zero_filled[county_df_filled.percent_white.isnull()]

Note that this will change some statistics! This is because we aren't ignoring the missing values anymore and are instead including them in the calculation.

In [ ]:
county_zero_filled.describe()